# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders` from the file `Oders.csv` located in the `data` folder into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# your code here
orders = pd.read_csv('../data/Orders.csv')
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
# your code here

,,amount_spent,Quantiles
CustomerID,Country,,
14646,Netherlands,280206.02,VIP
18102,United Kingdom,259657.30,VIP
17450,United Kingdom,194550.79,VIP
16446,United Kingdom,168472.50,VIP
14911,EIRE,143825.06,VIP
...,...,...,...
17956,United Kingdom,12.75,Casual
16454,United Kingdom,6.90,Casual
14792,United Kingdom,6.20,Casual


In [3]:
unique_customer = orders['CustomerID'].unique()

In [4]:
sum_amount_spent = []
for nom in unique_customer:
    sum_amount_spent.append(orders[orders["CustomerID"] == nom]['amount_spent'].sum())
sum_amount_spent

[5391.210000000001,
 3237.5400000000004,
 7281.379999999999,
 948.2499999999999,
 876.0,
 4668.300000000001,
 5630.870000000001,
 5411.91,
 60767.899999999994,
 2005.63,
 489.6,
 598.83,
 81024.84,
 389.44000000000005,
 6487.45,
 91062.38,
 711.86,
 79.6,
 28117.04,
 17220.36,
 2962.500000000001,
 3539.370000000001,
 832.88,
 14758.220000000001,
 192.60000000000005,
 3084.6800000000003,
 1659.75,
 2995.72,
 243.27999999999997,
 4204.1,
 683.13,
 399.51,
 233.45000000000005,
 689.9499999999999,
 3190.55,
 65039.619999999995,
 1475.02,
 7930.540000000001,
 277.35,
 21086.300000000003,
 566.5999999999999,
 4100.92,
 379.7300000000001,
 239.92,
 1008.5400000000003,
 313.49000000000007,
 33719.73,
 423.04,
 1607.0600000000004,
 226.75000000000009,
 1049.37,
 2043.0800000000002,
 3849.7800000000007,
 825.92,
 161.00000000000003,
 800.24,
 2575.0000000000005,
 13375.87,
 518.67,
 115.65000000000002,
 1198.23,
 2888.7500000000005,
 369.50000000000006,
 379.8299999999999,
 487.75,
 143825.06,
 

In [5]:
topcustomers = pd.DataFrame({'CustomerID':unique_customer, 'Aggregated_spent':sum_amount_spent,'Country':np.nan})
topcustomers.sort_values(by='Aggregated_spent',ascending = False, inplace=True)


countries = []
for i in topcustomers["CustomerID"]:
    countries.append(orders['Country'][orders['CustomerID'] == i].unique()[0])

topcustomers["Country"] = countries
topcustomers.index = list(range(4339))
topcustomers

,CustomerID,Aggregated_spent,Country
0,14646,280206.02,Netherlands
1,18102,259657.30,United Kingdom
2,17450,194550.79,United Kingdom
3,16446,168472.50,United Kingdom
4,14911,143825.06,EIRE
...,...,...,...
4334,17956,12.75,United Kingdom
4335,16454,6.90,United Kingdom
4336,14792,6.20,United Kingdom
4337,16738,3.75,United Kingdom


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [20]:
# your code here


Country
United Kingdom     178
Germany             11
France               9
Switzerland          3
EIRE                 2
Japan                2
Portugal             2
Spain                2
Norway               1
Netherlands          1
Singapore            1
Finland              1
Sweden               1
Denmark              1
Cyprus               1
Channel Islands      1
Australia            1
Name: Quantiles, dtype: int64

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [21]:
# your code here


Country
United Kingdom     934
Germany             39
France              29
Belgium             11
Switzerland          9
Spain                7
Portugal             7
Norway               7
Italy                5
Finland              5
Channel Islands      4
Australia            4
Japan                4
EIRE                 3
Denmark              3
Cyprus               3
Sweden               2
Israel               2
Lebanon              1
Iceland              1
Greece               1
Malta                1
Netherlands          1
Poland               1
Singapore            1
Canada               1
Austria              1
Name: Quantiles, dtype: int64